<span style="font-weight:bold; color:#4CAF50; font-size:28px;">
TASK 4 OPTIMIZATION MODEL</span>

<div style="font-family: Arial, sans-serif; text-align: left; background-color: #f2f2f2; padding: 10px; border-radius: 5px;">
    <span style="font-weight: bold; color: #4CAF50; font-size: 20px;">Author: Bisma yb</span><br>
    <span style="font-weight: bold; color: #FF9800; font-size: 18px;">Domain:Data Science</span><br>
</div>

In [3]:
pip install pulp


   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   -- ------------------------------------- 1.0/17.7 MB 4.6 MB/s eta 0:00:04
   ------- -------------------------------- 3.4/17.7 MB 8.1 MB/s eta 0:00:02
   ------------- -------------------------- 6.0/17.7 MB 9.7 MB/s eta 0:00:02
   ------------------- -------------------- 8.7/17.7 MB 10.7 MB/s eta 0:00:01
   -------------------------- ------------- 11.5/17.7 MB 11.5 MB/s eta 0:00:01
   -------------------------------- ------- 14.2/17.7 MB 11.7 MB/s eta 0:00:01
   ------------------------------------- -- 16.8/17.7 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 17.7/17.7 MB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pulp
import numpy as np
import pandas as pd

# Problem Setup: Vehicle Routing Problem (VRP)
# Define locations (including depot)
n_locations = 6  # Example: 5 customer locations + 1 depot
vehicles = 2  # Number of available vehicles
vehicle_capacity = 15  # Capacity constraint

# Distance matrix (symmetric, depot at index 0)
distance_matrix = np.array([
    [0, 10, 15, 20, 25, 30],
    [10, 0, 35, 25, 30, 40],
    [15, 35, 0, 30, 20, 50],
    [20, 25, 30, 0, 45, 35],
    [25, 30, 20, 45, 0, 55],
    [30, 40, 50, 35, 55, 0]
])

# Demand per location (excluding depot)
demands = [0, 5, 10, 8, 7, 6]

# Define the optimization model
model = pulp.LpProblem("Vehicle_Routing_Problem", pulp.LpMinimize)

# Decision Variables: x[i][j][v] -> 1 if vehicle v travels from i to j
x = pulp.LpVariable.dicts("x", ((i, j, v) for i in range(n_locations) for j in range(n_locations) for v in range(vehicles)), cat='Binary')

# Objective Function: Minimize total travel distance
model += pulp.lpSum(distance_matrix[i][j] * x[i, j, v] for i in range(n_locations) for j in range(n_locations) for v in range(vehicles))

# Constraints
# 1. Each customer is visited exactly once
for j in range(1, n_locations):
    model += pulp.lpSum(x[i, j, v] for i in range(n_locations) for v in range(vehicles) if i != j) == 1

# 2. Vehicle capacity constraint
for v in range(vehicles):
    model += pulp.lpSum(demands[j] * x[i, j, v] for i in range(n_locations) for j in range(1, n_locations) if i != j) <= vehicle_capacity

# 3. Vehicles must leave and return to the depot
for v in range(vehicles):
    model += pulp.lpSum(x[0, j, v] for j in range(1, n_locations)) == 1  # Start at depot
    model += pulp.lpSum(x[i, 0, v] for i in range(1, n_locations)) == 1  # Return to depot

# Solve the model
model.solve()

# Print results
print("Status:", pulp.LpStatus[model.status])
for v in range(vehicles):
    for i in range(n_locations):
        for j in range(n_locations):
            if pulp.value(x[i, j, v]) == 1:
                print(f"Vehicle {v} travels from {i} to {j}")

# Total distance
print("Total Distance:", pulp.value(model.objective))


Status: Infeasible
Vehicle 0 travels from 0 to 5
Vehicle 0 travels from 1 to 0
Vehicle 0 travels from 2 to 4
Vehicle 1 travels from 0 to 1
Vehicle 1 travels from 1 to 0
Vehicle 1 travels from 4 to 2
Total Distance: 125.0
